In [1]:
!pip install datasets evaluate seqeval -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-c

# Imports

In [2]:
import os
import torch
import torch.nn as nn
from datasets import load_dataset
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import evaluate
import time

# Load dataset

In [3]:
raw_datasets = load_dataset("conll2003", trust_remote_code=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [4]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

# Prepare data

In [5]:
teacher_checkpoint = "dslim/bert-large-NER"
student_checkpoint = "dslim/bert-base-NER"

In [6]:
t_tokenizer = AutoTokenizer.from_pretrained(teacher_checkpoint)
s_tokenizer = AutoTokenizer.from_pretrained(student_checkpoint)

tokenizer_config.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [7]:
t_tokenizer.vocab == s_tokenizer.vocab

True

In [8]:
tokenizer = AutoTokenizer.from_pretrained(teacher_checkpoint)

In [9]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [10]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [11]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [12]:
ner_feature = raw_datasets["train"].features["ner_tags"]
ner_feature

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [13]:
label_names = ner_feature.feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [14]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [27]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

eval_dataloader = DataLoader(
    tokenized_datasets["validation"], collate_fn=data_collator, batch_size=8
)

test_dataloader = DataLoader(
    tokenized_datasets["test"], collate_fn=data_collator, batch_size=8
)

In [16]:
id2label =  {
    0: "O",
    1: "B-MISC",
    2: "I-MISC",
    3: "B-PER",
    4: "I-PER",
    5: "B-ORG",
    6: "I-ORG",
    7: "B-LOC",
    8: "I-LOC"
  }

In [17]:
def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions

# Load model

In [32]:
teacher_model = AutoModelForTokenClassification.from_pretrained(
    teacher_checkpoint,
    device_map='cuda'
)

Some weights of the model checkpoint at dslim/bert-large-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [33]:
student_model = AutoModelForTokenClassification.from_pretrained(
    student_checkpoint,
    device_map='cuda'
)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [61]:
def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

In [62]:
print_size_of_model(teacher_model)

Size (MB): 1330.288643


In [63]:
print_size_of_model(student_model)

Size (MB): 430.973955


# Distillation

In [34]:
metric = evaluate.load("seqeval")

## Pre-evaluate

In [35]:
for batch in test_dataloader:
    with torch.no_grad():
        batch = {k: v.to('cuda') for k, v in batch.items()}
        outputs = teacher_model(**batch)

    predictions = outputs.logits.argmax(dim=-1)
    labels = batch["labels"]

    true_predictions, true_labels = postprocess(predictions, labels)
    metric.add_batch(predictions=true_predictions, references=true_labels)

results = metric.compute()

print(f'Teacher F1: {results["overall_f1"]:.3f}')

Teacher F1: 0.816


In [36]:
for batch in test_dataloader:
    with torch.no_grad():
        batch = {k: v.to('cuda') for k, v in batch.items()}
        outputs = student_model(**batch)

    predictions = outputs.logits.argmax(dim=-1)
    labels = batch["labels"]

    true_predictions, true_labels = postprocess(predictions, labels)
    metric.add_batch(predictions=true_predictions, references=true_labels)

results = metric.compute()

print(f'Student F1: {results["overall_f1"]:.3f}')

Student F1: 0.657


## Train

In [37]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(student_model.parameters(), lr=2e-5)
epochs = 3
T=2
soft_target_loss_weight = 0.25

In [38]:
teacher_model.eval()

best_model_state = None
best_f1 = -1

for epoch in range(epochs):
    student_model.train()
    train_loss = 0
    for batch in train_dataloader:
        batch = {k: v.to('cuda') for k, v in batch.items()}

        optimizer.zero_grad()

        with torch.no_grad():
            teacher_logits = teacher_model(**batch).logits

        student_logits = student_model(**batch).logits

        soft_targets = nn.functional.softmax(teacher_logits / T, dim=-1)
        soft_prob = nn.functional.log_softmax(student_logits / T, dim=-1)

        # soft_targets_loss = -torch.sum(soft_targets * soft_prob) / soft_prob.size()[0] * (T**2)
        soft_targets_loss = torch.sum(soft_targets * (soft_targets.log() - soft_prob)) / soft_prob.size()[0] * (T**2)

        label_loss = criterion(student_logits.permute(0, 2, 1), batch['labels'])

        loss = soft_target_loss_weight * soft_targets_loss + (1 - soft_target_loss_weight) * label_loss

        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    student_model.eval()
    for batch in eval_dataloader:
        with torch.no_grad():
            batch = {k: v.to('cuda') for k, v in batch.items()}
            outputs = student_model(**batch)

        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]

        true_predictions, true_labels = postprocess(predictions, labels)
        metric.add_batch(predictions=true_predictions, references=true_labels)

    results = metric.compute()

    if results["overall_f1"] > best_f1:
        best_model_state = student_model.state_dict()
        best_f1 = results["overall_f1"]

    print(f'Epoch: {epoch}, loss: {loss / len(train_dataloader):.3f}, F1: {results["overall_f1"]:.3f}')

Epoch: 0, loss: 0.001, F1: 0.867
Epoch: 1, loss: 0.002, F1: 0.849
Epoch: 2, loss: 0.000, F1: 0.846


# Post-evaluate

In [39]:
metric = evaluate.load("seqeval")

In [40]:
if best_model_state is not None:
    student_model.load_state_dict(best_model_state)

student_model.eval()
for batch in test_dataloader:
    with torch.no_grad():
        batch = {k: v.to('cuda') for k, v in batch.items()}
        outputs = student_model(**batch)

    predictions = outputs.logits.argmax(dim=-1)
    labels = batch["labels"]

    true_predictions, true_labels = postprocess(predictions, labels)
    metric.add_batch(predictions=true_predictions, references=true_labels)

results = metric.compute()

print(f'Student F1: {results["overall_f1"]:.3f}')

Student F1: 0.845


# Measure inference time

In [96]:
nlp = pipeline("ner", model=student_model, tokenizer=tokenizer, aggregation_strategy="simple")

start = time.time()
ner_results = nlp(' '.join(raw_datasets["test"][3]["tokens"]))
end = time.time()
print(ner_results)

Device set to use cuda


[{'entity_group': 'LOC', 'score': np.float32(0.99209493), 'word': 'Japan', 'start': 0, 'end': 5}, {'entity_group': 'MISC', 'score': np.float32(0.985772), 'word': 'Asian Cup', 'start': 33, 'end': 42}, {'entity_group': 'LOC', 'score': np.float32(0.9918749), 'word': 'Syria', 'start': 78, 'end': 83}, {'entity_group': 'MISC', 'score': np.float32(0.8937901), 'word': 'Group C', 'start': 89, 'end': 96}]


In [97]:
print(f'Inference time: {1000*(end - start)} ms')

Inference time: 12.894153594970703 ms
